In [50]:
from google.cloud import bigquery
import pandas as pd

# Authenticate with Google Cloud using a service account key
client = bigquery.Client()

In [51]:
def execute_query(query):
    # Execute the query
    query_job = client.query(query)
    # Fetch the results
    results = query_job.result()
    df = client.query(query).to_dataframe()
    return df

In [52]:
# create a complete_dataset table to be used as a base for further joins.
query = '''
DROP TABLE IF EXISTS `olist.complete_dataset`;
CREATE TABLE `olist.complete_dataset` AS (
    SELECT review_id,
        r.order_id,
        o.customer_id,
        o.order_status,
        o.order_purchase_timestamp,
        o.order_delivered_customer_date,
        o.order_estimated_delivery_date,
        o.order_approved_at,
        review_score,
        review_answer_timestamp,
        review_creation_date,
        review_interval_days,
        DATETIME_DIFF(CAST(o.order_delivered_customer_date AS DATETIME), CAST(o.order_estimated_delivery_date AS DATETIME), DAY) AS delivery_interval_days
    FROM (
        SELECT * FROM (
            SELECT review_id, order_id, review_score,
                ROW_NUMBER() OVER (PARTITION BY review_id ORDER BY review_id) AS review_ranking,
                DENSE_RANK() OVER (PARTITION BY order_id ORDER BY review_answer_timestamp DESC) AS order_ranking,
                review_answer_timestamp,
                review_creation_date,
                DATETIME_DIFF(CAST(review_answer_timestamp AS DATETIME), CAST(review_creation_date AS DATETIME), DAY) AS review_interval_days
            FROM `olist.order_reviews`
        ) WHERE review_ranking = 1 AND order_ranking = 1
        ORDER BY review_id, order_id
    ) AS r
    LEFT JOIN `olist.orders` AS o
    ON r.order_id = o.order_id
    WHERE order_approved_at IS NOT NULL AND order_purchase_timestamp IS NOT NULL AND order_delivered_customer_date IS NOT NULL AND order_estimated_delivery_date IS NOT NULL AND order_status != 'canceled' OR order_status != 'unavailable'
);

SELECT * FROM `olist.complete_dataset`
'''

print(f"Creating table olist.complete_dataset")
complete_dataset = execute_query(query)


Creating table olist.complete_dataset


In [53]:
complete_dataset.head()

,review_id,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,order_approved_at,review_score,review_answer_timestamp,review_creation_date,review_interval_days,delivery_interval_days
0,69ac6a27fde9855ebeaaecac0f78058b,809a282bbd5dbcabb6f2f724fca862ec,622e13439d6b5a0b486c435618b2679e,canceled,2016-09-13 15:24:19+00:00,NaT,2016-09-30 00:00:00+00:00,2016-10-07 13:16:46+00:00,1,2016-10-26 12:31:00,2016-10-02 00:00:00,24,<NA>
1,49f695dffa457eaba90d388a5c37e942,e5215415bb6f76fe3b7cb68103a0d1c0,b6f6cbfc126f1ae6723fe2f9b3751208,canceled,2016-10-22 08:25:27+00:00,NaT,2016-10-24 00:00:00+00:00,NaT,1,2016-10-11 14:31:29,2016-10-09 00:00:00,2,<NA>
2,cef1ee03ded4d6272894a2eead6e1328,2e7a8482f6fb09756ca50c10d7bfc047,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-09-04 21:15:19+00:00,NaT,2016-10-20 00:00:00+00:00,2016-10-07 13:18:03+00:00,1,2016-11-15 16:00:34,2016-10-22 00:00:00,24,<NA>
3,34d62feeefaf60ef6ff7204af19fe109,71303d7e93b399f5bcd537d124c0bcfa,b106b360fe2ef8849fbbd056f777b4d5,canceled,2016-10-02 22:07:52+00:00,NaT,2016-10-25 00:00:00+00:00,2016-10-06 15:50:56+00:00,1,2016-10-27 23:40:51,2016-10-27 00:00:00,0,<NA>
4,a93139d9d1314158c080e3db7e79618b,e5fa5a7210941f7d56d0208e4e071d35,683c54fc24d40ee9f8a6fc179fd9856c,canceled,2016-09-05 00:15:34+00:00,NaT,2016-10-28 00:00:00+00:00,2016-10-07 13:17:15+00:00,1,2016-10-30 01:47:48,2016-10-29 00:00:00,1,<NA>


In [54]:
# calculate the overall average review score
query = '''SELECT ROUND(AVG(review_score),2) AS avg_review_score FROM `olist.complete_dataset`'''
avg_review_score = execute_query(query)
avg_review_score

,avg_review_score
0,4.1


In [55]:
# calculate the % of reviews for delivered and non-delivered orders and % by review_score
query= '''SELECT review_score,
       COUNT(CASE WHEN order_status = 'delivered' THEN 1 ELSE NULL END) /
       (SELECT COUNT(*) FROM `olist.complete_dataset`) * 100 AS del_review_pct,
       COUNT(CASE WHEN order_status != 'delivered' THEN 1 ELSE NULL END) /
        (SELECT COUNT(*) FROM `olist.complete_dataset`) * 100 AS no_del_review_pct,
FROM `olist.complete_dataset`
GROUP BY review_score
ORDER BY review_score
'''
del_review_pct = execute_query(query)
del_review_pct

,review_score,del_review_pct,no_del_review_pct
0,1,9.494782,1.567070
1,2,2.971994,0.167277
2,3,8.057018,0.183697
3,4,19.290252,0.129306
4,5,57.926172,0.212432


In [56]:
# what proportion of on time delivered orders received reviews equal to or above 4?
query = '''SELECT review_score,
    (COUNT(review_score) / (SELECT COUNT(*) FROM `olist.complete_dataset`)) * 100 AS review_pct
FROM `olist.complete_dataset`
WHERE delivery_interval_days <= 0 AND
     order_purchase_timestamp < order_approved_at AND
    order_status = 'delivered' AND review_score >= 4
GROUP BY review_score
ORDER BY review_score'''

ontime_pos_del_review_pct = execute_query(query)
ontime_pos_del_review_pct

,review_score,review_pct
0,4,18.370740
1,5,56.109726


In [57]:
# what proportion of on time delivered orders received reviews below 4?
query = '''SELECT review_score,
    (COUNT(review_score) / (SELECT COUNT(*) FROM `olist.complete_dataset`)) * 100 AS review_pct
FROM `olist.complete_dataset`
WHERE delivery_interval_days <= 0 AND
    order_purchase_timestamp < order_approved_at AND
    order_status = 'delivered' AND review_score < 4
GROUP BY review_score
ORDER BY review_score'''

ontime_neg_review_pct = execute_query(query)
ontime_neg_review_pct

,review_score,review_pct
0,1,5.908069
1,2,2.380879
2,3,7.240130


In [58]:
# what proportion of late delivered orders received good reviews equal to or above 4?
query = '''SELECT review_score,
(COUNT(review_score) / (SELECT COUNT(*) FROM `olist.complete_dataset`)) * 100 AS review_pct
FROM `olist.complete_dataset`
WHERE delivery_interval_days > 0 AND
    order_purchase_timestamp < order_approved_at AND
    order_status = 'delivered' AND review_score >= 4
GROUP BY review_score
ORDER BY review_score'''

late_pos_review_pct = execute_query(query)
late_pos_review_pct

,review_score,review_pct
0,4,0.655768
1,5,1.066264


In [59]:
# what proportion of late delivered orders received bad reviews below 4?
query = '''SELECT review_score,
(COUNT(review_score) / (SELECT COUNT(*) FROM `olist.complete_dataset`)) * 100 AS review_pct
FROM `olist.complete_dataset`
WHERE delivery_interval_days > 0 AND
order_purchase_timestamp < order_approved_at AND
order_status = 'delivered' AND review_score < 4
GROUP BY review_score
ORDER BY review_score
'''

late_neg_review_pct = execute_query(query)
late_neg_review_pct

,review_score,review_pct
0,1,3.471773
1,2,0.558275
2,3,0.702975


In [60]:
# product category and review distribution
query = '''WITH product_reviews AS (
  SELECT * FROM (
    SELECT review_id,
`olist.translations`.string_field_0 AS product_category_name,
    `olist.translations`.string_field_1 AS product_category_name_english,
        `olist.complete_dataset`.order_id,
        customer_id,
        order_status,
        review_score
          FROM `olist.complete_dataset`
  LEFT JOIN `olist.order_items`
  ON `olist.complete_dataset`.order_id = `olist.order_items`.order_id
  LEFT JOIN `olist.products`
  ON `olist.order_items`.product_id = `olist.products`.product_id
  INNER JOIN `olist.translations`
  ON `olist.products`.product_category_name = `olist.translations`.string_field_0
  WHERE order_status = 'delivered'
  )
)

-- calculate the number of reviews for each score based on category
SELECT product_reviews.product_category_name_english,
review_score,
COUNT(*) AS product_review_count FROM product_reviews
GROUP BY product_reviews.product_category_name_english, review_score
ORDER BY product_reviews.product_category_name_english, review_score DESC'''

product_review = execute_query(query)
product_review


,product_category_name_english,review_score,product_review_count
0,agro_industry_and_commerce,5,108
1,agro_industry_and_commerce,4,57
2,agro_industry_and_commerce,3,13
3,agro_industry_and_commerce,2,7
4,agro_industry_and_commerce,1,21
...,...,...,...
334,watches_gifts,5,3319
335,watches_gifts,4,1106
336,watches_gifts,3,521
337,watches_gifts,2,202


In [61]:
# review scores by geographical region
query='''
DROP TABLE IF EXISTS `olist.review_geolocations`;
CREATE TABLE `olist.review_geolocations` AS (
  SELECT review_id,
        order_id,
        o.customer_id,
        order_status,
        order_purchase_timestamp, order_delivered_customer_date,
        review_score,
        c.customer_state,
        c.customer_unique_id
  FROM `olist.complete_dataset` AS o
  INNER JOIN `olist.customers` AS c
  ON o.customer_id = c.customer_id
  INNER JOIN `olist.geolocations` AS g
  ON g.geolocation_zip_code_prefix = c.customer_zip_code_prefix
);

-- review scores by state
SELECT customer_state, review_score, COUNT(review_score) AS review_count
FROM `olist.review_geolocations`
WHERE order_status = 'delivered'
GROUP BY customer_state, review_score
ORDER BY customer_state, review_score DESC
'''

geo_review = execute_query(query)
geo_review

,customer_state,review_score,review_count
0,AC,5,4031
1,AC,4,1672
2,AC,3,811
3,AC,2,422
4,AC,1,527
...,...,...,...
130,TO,5,8698
131,TO,4,4139
132,TO,3,1681
133,TO,2,815


In [63]:
# review_count by state, year and month
query='''SELECT
  customer_state AS state,
  EXTRACT(YEAR FROM order_delivered_customer_date) AS year,
  FORMAT_DATETIME("%B", TIMESTAMP_TRUNC(order_delivered_customer_date, MONTH)) AS month,
  CONCAT(FORMAT_DATETIME("%B", TIMESTAMP_TRUNC(order_delivered_customer_date, MONTH)), '-', CAST(EXTRACT(YEAR FROM order_delivered_customer_date) AS STRING)) AS year_month,
  review_score,
  COUNT(review_id) AS num_reviews
FROM `olist.review_geolocations`
GROUP BY
  customer_state,
  year,
  month,
  year_month,
  review_score
ORDER BY
  customer_state,
  year,
  month,
  review_score
  '''

geo_year_month_review = execute_query(query)
geo_year_month_review

,state,year,month,year_month,review_score,num_reviews
0,AC,<NA>,None,None,1,164
1,AC,2017,August,August-2017,1,164
2,AC,2017,August,August-2017,2,164
3,AC,2017,August,August-2017,5,410
4,AC,2017,December,December-2017,3,164
...,...,...,...,...,...,...
2399,TO,2018,March,March-2018,5,313
2400,TO,2018,May,May-2018,1,134
2401,TO,2018,May,May-2018,3,167
2402,TO,2018,May,May-2018,4,488


<!-- # avg_review_score.to_csv('avg_review_score.csv')
# del_review_pct.to_csv('del_review.csv')
# ontime_pos_del_review_pct.to_csv('ontime_pos_del_review_pct.csv')
# ontime_neg_review_pct.to_csv('ontime_neg_review_pct.csv')
# late_pos_review_pct.to_csv('late_pos_review_pct.csv')
# late_neg_review_pct.to_csv('late_neg_review_pct.csv')
# product_review.to_csv('product_review.csv')
# geo_review.to_csv('geo_review.csv')
# geo_year_month_review.to_csv('geo_year_month_review.csv')
# complete_dataset.to_csv('complete_dataset.csv') -->